In [1]:
import pandas as pd
import xarray as xr
from pathlib import Path

In [2]:
filename = Path('data', 'Heitronics-Apogee_Comparison_irt60s.dat')

### Here is the first 6 lines of the file:

"TOA5","Heitronics-Apogee Comparison","CR1000X","9292","CR1000X.Std.03.01","CPU:Apogee-Heitronics-An     alog.CR1X","46927","irt60s"
   2 "TIMESTAMP","RECORD","sfc_ir_temp_Avg","sfc_ir_temp_Std","sfc_a_ir_temp_Avg","sfc_a_ir_temp_Std","lo     gger_temp","logger_volt","logger_LiBat"
   3 "TS","RN","Kelvin","Kelvin","Kelvin","Kelvin","Celsius","Volt","Volt"
   4 "","","Avg","Std","Avg","Std","Smp","Smp","Smp"
   5 "2022-07-15 18:57:00",0,318.3,0.062,318.1,0.342,26.34,13.64,3.93
   6 "2022-07-15 18:58:00",1,318.1,0.347,317.7,1.452,26.38,13.64,3.931

We can read the file with no keywords and see what we get. It will do the best it can to read the file making some default assumptions. Not bad for knowing nothing about the data file. But there are some issues. So we need to understand what keywords we should use and the values for those keywords.

In [3]:
df = pd.read_csv(filename)
df

,TOA5,Heitronics-Apogee Comparison,CR1000X,9292,CR1000X.Std.03.01,CPU:Apogee-Heitronics-Analog.CR1X,46927,irt60s
TIMESTAMP,RECORD,sfc_ir_temp_Avg,sfc_ir_temp_Std,sfc_a_ir_temp_Avg,sfc_a_ir_temp_Std,logger_temp,logger_volt,logger_LiBat
TS,RN,Kelvin,Kelvin,Kelvin,Kelvin,Celsius,Volt,Volt
NaN,NaN,Avg,Std,Avg,Std,Smp,Smp,Smp
2022-07-15 18:57:00,0,318.3,0.062,318.1,0.342,26.34,13.64,3.93
2022-07-15 18:58:00,1,318.1,0.347,317.7,1.452,26.38,13.64,3.931
...,...,...,...,...,...,...,...,...
2022-07-20 22:38:00,7421,308.5,0.051,308.2,0.056,24.29,13.7,3.933
2022-07-20 22:39:00,7422,308.5,0.06,308.2,0.023,24.29,13.71,3.935
2022-07-20 22:40:00,7423,308.6,0.065,308.2,0.041,24.3,13.7,3.932
2022-07-20 22:41:00,7424,308.6,0.053,308.3,0.024,24.3,13.71,3.939


- Even though the delimiter was assumed correctly to be a comma we should be explicit.
- We should indicate what rows should be skipped. There is some metadata about the values that we can skip over.
- There is a column of dates and we can have Pandas do the work of parsing that for us.
- We should index on time so we can do that during the read in process.
- We can indicate what row contains the header names. Notice skiprows is applied first so the header is second row in the file, but we state it is the first row in the file after skiprows is applied.

In [4]:
df = pd.read_csv(filename, delimiter=',', skiprows=[0, 2, 3], header=0, parse_dates=[0], index_col=[0])

Since Pandas does not have a native place to hold the metadata we can convert the Pandas Dataframe to an Xarray Dataset and add the metadata to the DataArrays.

Before we do that we can clean up the index column name. CF convention indicates the dimension and coordinate name for time dimension should match. This allows Xarray tools to work correctly. Suggested name to use is 'time'.

In [5]:
df.index.name = 'time'  # Set the index name to 'time'
ds = df.to_xarray()  # Convert from Pandas Dataframe to Xarray Dataset
ds

<xarray.Dataset>
Dimensions:            (time: 7426)
Coordinates:
  * time               (time) datetime64[ns] 2022-07-15T18:57:00 ... 2022-07-...
Data variables:
    RECORD             (time) int64 0 1 2 3 4 5 ... 7421 7422 7423 7424 7425
    sfc_ir_temp_Avg    (time) float64 318.3 318.1 318.5 ... 308.6 308.6 308.7
    sfc_ir_temp_Std    (time) float64 0.062 0.347 0.201 ... 0.065 0.053 0.047
    sfc_a_ir_temp_Avg  (time) float64 318.1 317.7 318.1 ... 308.2 308.3 308.3
    sfc_a_ir_temp_Std  (time) float64 0.342 1.452 0.587 ... 0.041 0.024 0.028
    logger_temp        (time) float64 26.34 26.38 26.4 26.42 ... 24.3 24.3 24.3
    logger_volt        (time) float64 13.64 13.64 13.64 ... 13.7 13.71 13.67
    logger_LiBat       (time) float64 3.93 3.931 3.931 ... 3.932 3.939 3.937

Since the units, and long name are in the header of the .dat file we can read the .dat file as ASCII and apply those values to the DataArray attributes.

In [6]:
# We will use the with: block controller to automatically close the file when we exit the loop
with open(filename, 'r') as f:
    for ii, line in enumerate(f):  # Loop over each line of the file
        line = line.rstrip()  # Strip off any extra white spaces or returns from the strings.
        if ii == 0:  # Read first line
            long_name = line.split(',')  # Convert string into list of strings
        elif ii == 2:  # Read the thrid line
            units = line.split(',')  # Convert string into list of strings
        elif ii >= 4:  # After we get the header information exit loop.
            break

In [7]:
units = units[1: ]  # Units has TIMESTAMP units. But we made this the index so we need to drop first value.

In [8]:
for ii, var_name in enumerate(ds.data_vars):  # Loop over the variable names in the Dataset
    ds[var_name].attrs['long_name'] = long_name[ii]  # Set the long_name attribute
    ds[var_name].attrs['units'] = units[ii]  # Set the units attribute

In [9]:
ds['sfc_ir_temp_Avg']

<xarray.DataArray 'sfc_ir_temp_Avg' (time: 7426)>
array([318.3, 318.1, 318.5, ..., 308.6, 308.6, 308.7])
Coordinates:
  * time     (time) datetime64[ns] 2022-07-15T18:57:00 ... 2022-07-20T22:42:00
Attributes:
    long_name:  "Heitronics-Apogee Comparison"
    units:      "Kelvin"

The RECORD column is not needed so we can drop it from the Dataset. By waiting until the end of this process we do not need to accomidate the RECORD column in our loops. Often it is easier to do more work and then clean up at the end to ensure everyting aligns up correctly.

In [10]:
del ds['RECORD']
ds

<xarray.Dataset>
Dimensions:            (time: 7426)
Coordinates:
  * time               (time) datetime64[ns] 2022-07-15T18:57:00 ... 2022-07-...
Data variables:
    sfc_ir_temp_Avg    (time) float64 318.3 318.1 318.5 ... 308.6 308.6 308.7
    sfc_ir_temp_Std    (time) float64 0.062 0.347 0.201 ... 0.065 0.053 0.047
    sfc_a_ir_temp_Avg  (time) float64 318.1 317.7 318.1 ... 308.2 308.3 308.3
    sfc_a_ir_temp_Std  (time) float64 0.342 1.452 0.587 ... 0.041 0.024 0.028
    logger_temp        (time) float64 26.34 26.38 26.4 26.42 ... 24.3 24.3 24.3
    logger_volt        (time) float64 13.64 13.64 13.64 ... 13.7 13.71 13.67
    logger_LiBat       (time) float64 3.93 3.931 3.931 ... 3.932 3.939 3.937